In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import CTransformers
import streamlit as st
from langchain.chains import ConversationalRetrievalChain

In [12]:
pip install -U langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_community-0.0.24-py3-none-any.whl.metadata (8.1 kB)
  Using cached orjson-3.9.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
Using cached langchain_community-0.0.24-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 5.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.8 MB/s eta 0:00:00
Using cached orjson-3.9.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (138 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.20
    Uninstalling langchain-commu

#### DirectoryLoader: This is likely a class or function that loads documents from a specified directory.
##### glob="*.pdf": This parameter specifies that only files with a .pdf extension should be considered for loading.
##### PyPDFLoader: This parameter specifies the loader class to be used for loading the documents

In [3]:
#load the pdf
loader = DirectoryLoader("/mnt/d/WSL/Medical-chatbot-using-LLMA2/Data/",glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

##### RecursiveCharacterTextSplitter: This is likely a class or function responsible for splitting text into chunks

In [4]:
#split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size =500,chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)  #object to split the text from the documents into chunks

##### To generate embeddings for your text data.

In [5]:
#Create Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs={'device':'cpu'})

/home/souvik/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### FAISS is a library for similarity search and clustering that is optimized for high-dimensional vectors.
##### It is commonly used for tasks like searching for similar vectors or clustering vectors based on their similarity.

In [6]:
#vector store
vector_store =  FAISS.from_documents(text_chunks,embeddings)


##### CTransformersallows LLMs to run on consumer hardware with effective CPU inferencing.

In [7]:
#Create LLM
llm = CTransformers(model="/mnt/d/WSL/Medical-chatbot-using-LLMA2/llama-2-7b-chat.ggmlv3.q4_K_S.bin",
                    model_type = "llama",
                    config={'max_new_tokens':128,"temperature":0.01}
                    )

#### LLMA2 optimized for dialogue use cases.

In [8]:
#memory
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

##### This chain takes in chat history (a list of messages) and new questions, and then returns an answer to that question.

In [9]:
#k=2 will retrive the top 2 answers from FAISS vector database
chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type='stuff',
                                              retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                              memory=memory)


In [10]:
st.title("Healthcare Chatbot 👩🏻‍⚕️")

2024-03-01 09:49:50.961 
  command:

    streamlit run /home/souvik/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()